### Word Group In Youtube Sentence

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 28  # native word end index

# sentence ratio and time shift
adjust_sent_word_ratio = 50
adjust_text_word_ratio = 40
shift = 0.3  # sentence time shift

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")


6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/2-Word Group In Youtube Sentence/{lang_folder.capitalize()} {lang_pair.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for i in max(all_index_list, key=len):
        word = words[i]
        text_list.append(word)
    text = " ".join(text_list)
    
    return text 

In [7]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [8]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988210,karneleme,5
988211,karnaya,5
988212,dörtlümüzün,5
988213,karnavalınız,5


In [9]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [10]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [11]:
df_pair = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main,english_word
0,abaküs,abacus
1,abluka,blockade
2,absorbe,absorb
3,absürt,absurd
4,açelya,azalea
...,...,...
1773,zebra,zebra
1774,zikzak,zigzag
1775,zombi,zombie
1776,zooloji,zoology


In [12]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair = pd.merge(df_pair,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair.drop_duplicates(inplace=True)
    df_pair.reset_index(drop=True, inplace=True)
    df_pair = df_pair.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair.rename(columns={"word":"dict_entry_main"}, inplace=True)
else:
    pass

if etymology_word:
    df_pair
else:
    df_pair = df_pair.head(0)
    
df_pair

,dict_entry_main,english_word


In [13]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [14]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [15]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))

#### Youtube Sentence Analysis

In [65]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [66]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [67]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...,...
3063204,3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [68]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [69]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,33.4,"bunlar, kamyon, özgür, kullanılıyor, araçlar, ...","daha, için, büyük"
1,1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"arka, kolay, tarafı, bağlamak, burcu, olmayacak",
2,2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"tur, özgür, attıracağım, arabaya, yarım",
3,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,66.8,burcu,"biraz, daha, oldu, et, tamam, devam"
4,4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,22.3,"şimdilik, takacağız, artarsa, burcu, kar, kalı...","ama, iki"
...,...,...,...,...,...,...,...,...
3063204,3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA,9.2,"köşesindeki, kartlara, oynatma, linkini, üst, ...",ve
3063205,3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA,9.2,"bunlardan, uyarla, kenar, ekledim, yapabilirsi...",bu
3063206,3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA,20.1,"örgü, elle, dek, sonraki, görüşene, farklı, ib...","bir, lütfen, olan"
3063207,3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA,27.4,"beğenmeyi, hoşçakalın, videomu, kanalıma, unut...","iyi, olarak, çok"


In [70]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,33.4,"bunlar, kamyon, özgür, kullanılıyor, araçlar, ...","daha, için, büyük"
1,1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"arka, kolay, tarafı, bağlamak, burcu, olmayacak",NaN
2,2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"tur, özgür, attıracağım, arabaya, yarım",NaN
3,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,66.8,burcu,"biraz, daha, oldu, et, tamam, devam"
4,4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,22.3,"şimdilik, takacağız, artarsa, burcu, kar, kalı...","ama, iki"
...,...,...,...,...,...,...,...,...
3063204,3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA,9.2,"köşesindeki, kartlara, oynatma, linkini, üst, ...",ve
3063205,3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA,9.2,"bunlardan, uyarla, kenar, ekledim, yapabilirsi...",bu
3063206,3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA,20.1,"örgü, elle, dek, sonraki, görüşene, farklı, ib...","bir, lütfen, olan"
3063207,3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA,27.4,"beğenmeyi, hoşçakalın, videomu, kanalıma, unut...","iyi, olarak, çok"


In [71]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,66.8,burcu,"biraz, daha, oldu, et, tamam, devam"
1,8,248.832,252.736,çünkü durum şu durumu göstereyim size,8V9tq1pe8eI,50.1,"durum, durumu, göstereyim","çünkü, şu, size"
2,9,254.325,256.668,bu şekilde bir dakika özgür,8V9tq1pe8eI,80.1,özgür,"bir, bu, dakika, şekilde"
3,14,308.735,312.000,özgür nehircim bana yardım eder misin,8V9tq1pe8eI,50.1,"özgür, eder, nehircim","yardım, misin, bana"
4,20,379.391,381.437,burcu çok güzel görünüyor,8V9tq1pe8eI,50.1,"burcu, görünüyor","çok, güzel"
...,...,...,...,...,...,...,...,...
989483,3063137,499.645,502.335,çok hızlanacak çünkü dediğim gibi her şeye bir...,7HrxKm4FDw8,55.7,"anda, şeye, hızlanacak, dediğim","bir, gibi, çok, çünkü, her"
989484,3063156,65.960,74.150,sonra,MvWp9pWLihA,100.1,NaN,sonra
989485,3063159,91.970,98.270,birlikte toplam,MvWp9pWLihA,50.1,toplam,birlikte
989486,3063191,503.190,513.210,bu birinci ve hemen altındaki iki ve hemen yan...,MvWp9pWLihA,50.1,"birinci, yanındaki, altındaki","ve, bu, üç, iki, hemen"


In [72]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
0,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,66.8,burcu,"biraz, daha, oldu, et, tamam, devam",biraz daha devam et devam et tamam oldu
1,8,248.832,252.736,çünkü durum şu durumu göstereyim size,8V9tq1pe8eI,50.1,"durum, durumu, göstereyim","çünkü, şu, size",çünkü
2,9,254.325,256.668,bu şekilde bir dakika özgür,8V9tq1pe8eI,80.1,özgür,"bir, bu, dakika, şekilde",bu şekilde bir dakika
3,14,308.735,312.000,özgür nehircim bana yardım eder misin,8V9tq1pe8eI,50.1,"özgür, eder, nehircim","yardım, misin, bana",bana yardım
4,20,379.391,381.437,burcu çok güzel görünüyor,8V9tq1pe8eI,50.1,"burcu, görünüyor","çok, güzel",çok güzel
...,...,...,...,...,...,...,...,...,...
989483,3063137,499.645,502.335,çok hızlanacak çünkü dediğim gibi her şeye bir...,7HrxKm4FDw8,55.7,"anda, şeye, hızlanacak, dediğim","bir, gibi, çok, çünkü, her",gibi her
989484,3063156,65.960,74.150,sonra,MvWp9pWLihA,100.1,NaN,sonra,sonra
989485,3063159,91.970,98.270,birlikte toplam,MvWp9pWLihA,50.1,toplam,birlikte,birlikte
989486,3063191,503.190,513.210,bu birinci ve hemen altındaki iki ve hemen yan...,MvWp9pWLihA,50.1,"birinci, yanındaki, altındaki","ve, bu, üç, iki, hemen",iki ve hemen


In [73]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,biraz daha devam et devam et tamam oldu,197.690889,201.338000,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
1,çünkü,248.832000,249.465081,çünkü durum şu durumu göstereyim size,8V9tq1pe8eI
2,bu şekilde bir dakika,254.325000,256.234111,bu şekilde bir dakika özgür,8V9tq1pe8eI
3,bana yardım,309.970405,311.117568,özgür nehircim bana yardım eder misin,8V9tq1pe8eI
4,çok güzel,379.800200,380.700440,burcu çok güzel görünüyor,8V9tq1pe8eI
...,...,...,...,...,...
989483,gibi her,501.121863,501.649314,çok hızlanacak çünkü dediğim gibi her şeye bir...,7HrxKm4FDw8
989484,sonra,65.960000,74.150000,sonra,MvWp9pWLihA
989485,birlikte,91.970000,95.750000,birlikte toplam,MvWp9pWLihA
989486,iki ve hemen,508.473273,511.023818,bu birinci ve hemen altındaki iki ve hemen yan...,MvWp9pWLihA


In [74]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,biraz daha devam et devam et tamam oldu,197.390889,201.638000,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
1,çünkü,248.532000,249.765081,çünkü durum şu durumu göstereyim size,8V9tq1pe8eI
2,bu şekilde bir dakika,254.025000,256.534111,bu şekilde bir dakika özgür,8V9tq1pe8eI
3,bana yardım,309.670405,311.417568,özgür nehircim bana yardım eder misin,8V9tq1pe8eI
4,çok güzel,379.500200,381.000440,burcu çok güzel görünüyor,8V9tq1pe8eI
...,...,...,...,...,...
989483,gibi her,500.821863,501.949314,çok hızlanacak çünkü dediğim gibi her şeye bir...,7HrxKm4FDw8
989484,sonra,65.660000,74.450000,sonra,MvWp9pWLihA
989485,birlikte,91.670000,96.050000,birlikte toplam,MvWp9pWLihA
989486,iki ve hemen,508.173273,511.323818,bu birinci ve hemen altındaki iki ve hemen yan...,MvWp9pWLihA


In [75]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,biraz daha devam et devam et tamam oldu,197,202,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
1,çünkü,249,250,çünkü durum şu durumu göstereyim size,8V9tq1pe8eI
2,bu şekilde bir dakika,254,257,bu şekilde bir dakika özgür,8V9tq1pe8eI
3,bana yardım,310,311,özgür nehircim bana yardım eder misin,8V9tq1pe8eI
4,çok güzel,380,381,burcu çok güzel görünüyor,8V9tq1pe8eI
...,...,...,...,...,...
989483,gibi her,501,502,çok hızlanacak çünkü dediğim gibi her şeye bir...,7HrxKm4FDw8
989484,sonra,66,74,sonra,MvWp9pWLihA
989485,birlikte,92,96,birlikte toplam,MvWp9pWLihA
989486,iki ve hemen,508,511,bu birinci ve hemen altındaki iki ve hemen yan...,MvWp9pWLihA


In [76]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE
3,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw
4,yok yok o kadar yok yok o kadar yok ama şimdi ...,2162,2168,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg
...,...,...,...,...,...
989483,o,250,251,o elbise ne kadardı çünkü uçmayayım çok,LLodk18CmHU
989484,o,7941,7942,beyrutta o,Y5pjN6z1JjU
989485,o,267,267,o çantanın dakika sol,v_ZhT1JJe4g
989486,o,2191,2192,o çıkıyor,sDR1xwXjMuE


In [77]:
df_word_group_time_loc_in_sent_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_sent_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_sent_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...
3,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s
4,yok yok o kadar yok yok o kadar yok ama şimdi ...,2162,2168,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg,https://www.youtube.com/watch?v=DbsR_Cccadg&t=...
...,...,...,...,...,...,...
989483,o,250,251,o elbise ne kadardı çünkü uçmayayım çok,LLodk18CmHU,https://www.youtube.com/watch?v=LLodk18CmHU&t=...
989484,o,7941,7942,beyrutta o,Y5pjN6z1JjU,https://www.youtube.com/watch?v=Y5pjN6z1JjU&t=...
989485,o,267,267,o çantanın dakika sol,v_ZhT1JJe4g,https://www.youtube.com/watch?v=v_ZhT1JJe4g&t=...
989486,o,2191,2192,o çıkıyor,sDR1xwXjMuE,https://www.youtube.com/watch?v=sDR1xwXjMuE&t=...


In [ ]:
#df_word_group_time_loc_in_sent_result.to_excel("Word_Group_In_Youtube_Sentence_Test.xlsx", index=False)

#### Youtube Videoid Text Analysis

In [16]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [17]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [18]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [19]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

,index,video_id,sentence
0,0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...
1,1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...
2,2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...
3,3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...
4,4,00fr8yyBQYo,bill gates and george soros developed a new co...
...,...,...,...
18361,18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...
18362,18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...
18363,18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...
18364,18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...


In [20]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

,video_id,start_time
0,007uL_OBtPE,6.680
1,00Dl6tNNlZ8,36.803
2,00QJHZSg0UE,0.440
3,00b3ov_NFSI,0.000
4,00fr8yyBQYo,18.000
...,...,...
18361,zzOL9mRRU0c,9.333
18362,zzWQXUKr9OA,1.896
18363,zzXTIIo7EvM,36.932
18364,zzZfnY7Zjf0,0.000


In [21]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

,video_id,end_time
0,007uL_OBtPE,136.640
1,00Dl6tNNlZ8,813.903
2,00QJHZSg0UE,27.460
3,00b3ov_NFSI,42.400
4,00fr8yyBQYo,413.000
...,...,...
18361,zzOL9mRRU0c,533.254
18362,zzWQXUKr9OA,27.969
18363,zzXTIIo7EvM,1591.680
18364,zzZfnY7Zjf0,15.000


In [22]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

,video_id,start_time,end_time
0,007uL_OBtPE,6.680,136.640
1,00Dl6tNNlZ8,36.803,813.903
2,00QJHZSg0UE,0.440,27.460
3,00b3ov_NFSI,0.000,42.400
4,00fr8yyBQYo,18.000,413.000
...,...,...,...
18361,zzOL9mRRU0c,9.333,533.254
18362,zzWQXUKr9OA,1.896,27.969
18363,zzXTIIo7EvM,36.932,1591.680
18364,zzZfnY7Zjf0,0.000,15.000


In [23]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

,index,video_id,sentence,start_time,end_time
0,0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,6.680,136.640
1,1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,36.803,813.903
2,2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,0.440,27.460
3,3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,0.000,42.400
4,4,00fr8yyBQYo,bill gates and george soros developed a new co...,18.000,413.000
...,...,...,...,...,...
18361,18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,9.333,533.254
18362,18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,1.896,27.969
18363,18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,36.932,1591.680
18364,18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,0.000,15.000


In [24]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list2 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list2.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [25]:
df_text_ratio_result = pd.DataFrame(list(result_list2), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

,video_id,sentence,word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,20.9,"geçiyorum, yapılışına, pişmeye, ezine, soğumay...","üç, şu, olsun, istiyorum, bunu, zaman, bugün, ..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,5.9,"bilgiler, vardırı, paraya, manavgatta, sı, hız...","size, iyi, onlar, daha, böyle, teşekkür, bugün..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,22.3,"vakit, teklifimdeni, ölürümi, iselvii, ömürlük...","sen, ben, ne, şu, dur, benim, de, yapıyorsun, ..."
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,33.8,"farkında, kocam, barışırsınız, olsa, diyorsun,...","onu, bir, benim, değil, peki, senin, nasıl, ge..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.1,"schwab, your, whom, here, coronavirus, well, a...",
...,...,...,...,...,...
18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,6.9,"kalsiyum, kütür, yetiştiriliyorum, mide, havza...","son, bakalım, güzel, tüm, iyi, doğru, bir, hay..."
18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,27.4,"bırak, lan, kızınız, elimden, uzaklaş, nermin,...","olan, en, sana, da, gibi, ben, yoksa, bu, bana..."
18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,4.5,"akşam, den, dekoratif, bahsetmek, evin, hatta,...","belki, size, bile, oluyor, olsun, daha, böyle,..."
18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,23.4,"geçiş, memnunum, bakabileceğim, belgesellere, ...","kadar, çok, ben, bir, ve, küçük, büyük, için, ..."


In [26]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result 

,video_id,sentence,word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,20.9,"geçiyorum, yapılışına, pişmeye, ezine, soğumay...","üç, şu, olsun, istiyorum, bunu, zaman, bugün, ..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,5.9,"bilgiler, vardırı, paraya, manavgatta, sı, hız...","size, iyi, onlar, daha, böyle, teşekkür, bugün..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,22.3,"vakit, teklifimdeni, ölürümi, iselvii, ömürlük...","sen, ben, ne, şu, dur, benim, de, yapıyorsun, ..."
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,33.8,"farkında, kocam, barışırsınız, olsa, diyorsun,...","onu, bir, benim, değil, peki, senin, nasıl, ge..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.1,"schwab, your, whom, here, coronavirus, well, a...",NaN
...,...,...,...,...,...
18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,6.9,"kalsiyum, kütür, yetiştiriliyorum, mide, havza...","son, bakalım, güzel, tüm, iyi, doğru, bir, hay..."
18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,27.4,"bırak, lan, kızınız, elimden, uzaklaş, nermin,...","olan, en, sana, da, gibi, ben, yoksa, bu, bana..."
18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,4.5,"akşam, den, dekoratif, bahsetmek, evin, hatta,...","belki, size, bile, oluyor, olsun, daha, böyle,..."
18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,23.4,"geçiş, memnunum, bakabileceğim, belgesellere, ...","kadar, çok, ben, bir, ve, küçük, büyük, için, ..."


In [27]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

,video_id,sentence,word_ratio,different_word,intersect_word
0,0RIp8JRHXXE,kimsin sen selvi yüreğimin tetiğini indiren ki...,45.9,"yanıyor, damga, kim, burası, yüreğimin, tetiği...","en, sen, neden, çok, da, bir, her, bana, benim..."
1,0dAFdqL12F0,vazgeçmiyor kızım senden vazgeçmiyor fulya sen...,46.3,"sevdim, bebeğin, söyleyeceklerimden, diyorsun,...","güzel, bir, şeyler, benim, senin, olmak, ya, m..."
2,1LMDmZNFyv8,heya son zamanlarda epey meşulmüşsün ha kötü s...,43.4,"olursanız, öldürüyoz, napim, zamanlarda, geçi,...","son, bir, bile, pekala, iyi, musun, daha, zama..."
3,1a2_P8fAdmc,tamam efendim ustam bizim hesap ne kadar bir b...,44.5,"ek, tabi, e, bizce, paketler, çıkarmayan, para...","güzel, tüm, bir, böyle, efendim, tamam, buna, ..."
4,1jrLF7RNI0k,size benim güzel bir haberim var artık anneniz...,42.1,"faziletten, özel, foxta, haberim, parayı, akşa...","güzel, size, bir, kimse, benim, senin, biri, t..."
...,...,...,...,...,...
299,yWHodncIIqM,destek için abone ol beğen ve yorum yap lütfen,44.5,"beğen, abone, yorum, destek, yap","ve, lütfen, ol, için"
300,yqERJ6yqYRw,abone olur musun,66.8,abone,"olur, musun"
301,ytqhB6dpsnw,resimli güzel ve anlamlı sözler,40.1,"anlamlı, sözler, resimli","ve, güzel"
302,yvKCHSKRmbs,doğru nefes alıyor musun nefesini nerede hisse...,43.0,"alıyor, hissediyorsun, nefes, nefesini","nerede, doğru, musun"


In [28]:
df_adjust_text_ratio.loc[:,"search_string"] = df_adjust_text_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_text_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,video_id,sentence,word_ratio,different_word,intersect_word,search_string
0,0RIp8JRHXXE,kimsin sen selvi yüreğimin tetiğini indiren ki...,45.9,"yanıyor, damga, kim, burası, yüreğimin, tetiği...","en, sen, neden, çok, da, bir, her, bana, benim...",bir şey oldu neden bana
1,0dAFdqL12F0,vazgeçmiyor kızım senden vazgeçmiyor fulya sen...,46.3,"sevdim, bebeğin, söyleyeceklerimden, diyorsun,...","güzel, bir, şeyler, benim, senin, olmak, ya, m...",olmak çok güzel benim sana
2,1LMDmZNFyv8,heya son zamanlarda epey meşulmüşsün ha kötü s...,43.4,"olursanız, öldürüyoz, napim, zamanlarda, geçi,...","son, bir, bile, pekala, iyi, musun, daha, zama...",mi herkes iyi biri olabilir mi eğer bir
3,1a2_P8fAdmc,tamam efendim ustam bizim hesap ne kadar bir b...,44.5,"ek, tabi, e, bizce, paketler, çıkarmayan, para...","güzel, tüm, bir, böyle, efendim, tamam, buna, ...",de olmaz bu yüzden
4,1jrLF7RNI0k,size benim güzel bir haberim var artık anneniz...,42.1,"faziletten, özel, foxta, haberim, parayı, akşa...","güzel, size, bir, kimse, benim, senin, biri, t...",olur mu tamam siz çok
...,...,...,...,...,...,...
299,yWHodncIIqM,destek için abone ol beğen ve yorum yap lütfen,44.5,"beğen, abone, yorum, destek, yap","ve, lütfen, ol, için",için
300,yqERJ6yqYRw,abone olur musun,66.8,abone,"olur, musun",olur musun
301,ytqhB6dpsnw,resimli güzel ve anlamlı sözler,40.1,"anlamlı, sözler, resimli","ve, güzel",güzel ve
302,yvKCHSKRmbs,doğru nefes alıyor musun nefesini nerede hisse...,43.0,"alıyor, hissediyorsun, nefes, nefesini","nerede, doğru, musun",doğru


In [29]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence","word_ratio"]]
df_adjust_text_ratio

,search_string,video_id,sentence,word_ratio
0,bir şey oldu neden bana,0RIp8JRHXXE,kimsin sen selvi yüreğimin tetiğini indiren ki...,45.9
1,olmak çok güzel benim sana,0dAFdqL12F0,vazgeçmiyor kızım senden vazgeçmiyor fulya sen...,46.3
2,mi herkes iyi biri olabilir mi eğer bir,1LMDmZNFyv8,heya son zamanlarda epey meşulmüşsün ha kötü s...,43.4
3,de olmaz bu yüzden,1a2_P8fAdmc,tamam efendim ustam bizim hesap ne kadar bir b...,44.5
4,olur mu tamam siz çok,1jrLF7RNI0k,size benim güzel bir haberim var artık anneniz...,42.1
...,...,...,...,...
299,için,yWHodncIIqM,destek için abone ol beğen ve yorum yap lütfen,44.5
300,olur musun,yqERJ6yqYRw,abone olur musun,66.8
301,güzel ve,ytqhB6dpsnw,resimli güzel ve anlamlı sözler,40.1
302,doğru,yvKCHSKRmbs,doğru nefes alıyor musun nefesini nerede hisse...,43.0


In [30]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [31]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

,search_string,video_id,sentence,word_ratio,start_time,end_time
0,bir şey oldu neden bana,0RIp8JRHXXE,kimsin sen selvi yüreğimin tetiğini indiren ki...,45.9,0.220,18.580
1,olmak çok güzel benim sana,0dAFdqL12F0,vazgeçmiyor kızım senden vazgeçmiyor fulya sen...,46.3,0.460,50.520
2,mi herkes iyi biri olabilir mi eğer bir,1LMDmZNFyv8,heya son zamanlarda epey meşulmüşsün ha kötü s...,43.4,11.000,35.029
3,de olmaz bu yüzden,1a2_P8fAdmc,tamam efendim ustam bizim hesap ne kadar bir b...,44.5,2.320,40.000
4,olur mu tamam siz çok,1jrLF7RNI0k,size benim güzel bir haberim var artık anneniz...,42.1,0.000,48.800
...,...,...,...,...,...,...
299,için,yWHodncIIqM,destek için abone ol beğen ve yorum yap lütfen,44.5,0.000,324.600
300,olur musun,yqERJ6yqYRw,abone olur musun,66.8,0.001,182.000
301,güzel ve,ytqhB6dpsnw,resimli güzel ve anlamlı sözler,40.1,2.033,7.999
302,doğru,yvKCHSKRmbs,doğru nefes alıyor musun nefesini nerede hisse...,43.0,0.000,2.000


In [32]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,bir şey oldu neden bana,13.768414,16.933931,kimsin sen selvi yüreğimin tetiğini indiren ki...,0RIp8JRHXXE
1,olmak çok güzel benim sana,30.918193,35.140120,vazgeçmiyor kızım senden vazgeçmiyor fulya sen...,0dAFdqL12F0
2,mi herkes iyi biri olabilir mi eğer bir,21.476058,24.479683,heya son zamanlarda epey meşulmüşsün ha kötü s...,1LMDmZNFyv8
3,de olmaz bu yüzden,16.759416,19.509781,tamam efendim ustam bizim hesap ne kadar bir b...,1a2_P8fAdmc
4,olur mu tamam siz çok,13.971370,16.266667,size benim güzel bir haberim var artık anneniz...,1jrLF7RNI0k
...,...,...,...,...,...
299,için,42.339130,84.678261,destek için abone ol beğen ve yorum yap lütfen,yWHodncIIqM
300,olur musun,56.875687,182.000000,abone olur musun,yqERJ6yqYRw
301,güzel ve,3.380161,5.304677,resimli güzel ve anlamlı sözler,ytqhB6dpsnw
302,doğru,0.000000,0.222222,doğru nefes alıyor musun nefesini nerede hisse...,yvKCHSKRmbs


In [33]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,bir şey oldu neden bana,13.468414,17.233931,kimsin sen selvi yüreğimin tetiğini indiren ki...,0RIp8JRHXXE
1,olmak çok güzel benim sana,30.618193,35.440120,vazgeçmiyor kızım senden vazgeçmiyor fulya sen...,0dAFdqL12F0
2,mi herkes iyi biri olabilir mi eğer bir,21.176058,24.779683,heya son zamanlarda epey meşulmüşsün ha kötü s...,1LMDmZNFyv8
3,de olmaz bu yüzden,16.459416,19.809781,tamam efendim ustam bizim hesap ne kadar bir b...,1a2_P8fAdmc
4,olur mu tamam siz çok,13.671370,16.566667,size benim güzel bir haberim var artık anneniz...,1jrLF7RNI0k
...,...,...,...,...,...
299,için,42.039130,84.978261,destek için abone ol beğen ve yorum yap lütfen,yWHodncIIqM
300,olur musun,56.575688,182.300000,abone olur musun,yqERJ6yqYRw
301,güzel ve,3.080161,5.604677,resimli güzel ve anlamlı sözler,ytqhB6dpsnw
302,doğru,-0.300000,0.522222,doğru nefes alıyor musun nefesini nerede hisse...,yvKCHSKRmbs


In [34]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,bir şey oldu neden bana,13,17,kimsin sen selvi yüreğimin tetiğini indiren ki...,0RIp8JRHXXE
1,olmak çok güzel benim sana,31,35,vazgeçmiyor kızım senden vazgeçmiyor fulya sen...,0dAFdqL12F0
2,mi herkes iyi biri olabilir mi eğer bir,21,25,heya son zamanlarda epey meşulmüşsün ha kötü s...,1LMDmZNFyv8
3,de olmaz bu yüzden,16,20,tamam efendim ustam bizim hesap ne kadar bir b...,1a2_P8fAdmc
4,olur mu tamam siz çok,14,17,size benim güzel bir haberim var artık anneniz...,1jrLF7RNI0k
...,...,...,...,...,...
299,için,42,85,destek için abone ol beğen ve yorum yap lütfen,yWHodncIIqM
300,olur musun,57,182,abone olur musun,yqERJ6yqYRw
301,güzel ve,3,6,resimli güzel ve anlamlı sözler,ytqhB6dpsnw
302,doğru,0,1,doğru nefes alıyor musun nefesini nerede hisse...,yvKCHSKRmbs


In [35]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,ama benimle buraya kadar geldi sen şimdi git b...,17,37,ablan burada nerede i çeriye kadar gelemedi am...,NXB1VHHqPDE
1,biliyor musun kimse yok mu bana senin gibi,9,15,kimin nereye gideceğine ben karar veririm baze...,pvui4au_dYo
2,için ne demek biz seninle daha fazla zaman,38,43,semiha hanım edacığım ben de size bakıyordum k...,UiykPdDkFFY
3,ne oldu gerçekten bilmiyorum ne olduğunu,17,21,bakın bu çok riskli bir ameliyat bunu yalnızca...,QR_n_qdxGmE
4,mi herkes iyi biri olabilir mi eğer bir,21,25,heya son zamanlarda epey meşulmüşsün ha kötü s...,1LMDmZNFyv8
...,...,...,...,...,...
299,ol,1,2,abpne ol,7OvMvfbcTzk
300,ol,6,7,abone ol,7VbJZLq5Zko
301,ol,771,1234,abone ol,U2lLaxqugbU
302,ol,6,7,abone ol,V9Z155H7y7g


In [36]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,ama benimle buraya kadar geldi sen şimdi git b...,17,37,ablan burada nerede i çeriye kadar gelemedi am...,NXB1VHHqPDE,https://www.youtube.com/watch?v=NXB1VHHqPDE&t=17s
1,biliyor musun kimse yok mu bana senin gibi,9,15,kimin nereye gideceğine ben karar veririm baze...,pvui4au_dYo,https://www.youtube.com/watch?v=pvui4au_dYo&t=9s
2,için ne demek biz seninle daha fazla zaman,38,43,semiha hanım edacığım ben de size bakıyordum k...,UiykPdDkFFY,https://www.youtube.com/watch?v=UiykPdDkFFY&t=38s
3,ne oldu gerçekten bilmiyorum ne olduğunu,17,21,bakın bu çok riskli bir ameliyat bunu yalnızca...,QR_n_qdxGmE,https://www.youtube.com/watch?v=QR_n_qdxGmE&t=17s
4,mi herkes iyi biri olabilir mi eğer bir,21,25,heya son zamanlarda epey meşulmüşsün ha kötü s...,1LMDmZNFyv8,https://www.youtube.com/watch?v=1LMDmZNFyv8&t=21s
...,...,...,...,...,...,...
299,ol,1,2,abpne ol,7OvMvfbcTzk,https://www.youtube.com/watch?v=7OvMvfbcTzk&t=1s
300,ol,6,7,abone ol,7VbJZLq5Zko,https://www.youtube.com/watch?v=7VbJZLq5Zko&t=6s
301,ol,771,1234,abone ol,U2lLaxqugbU,https://www.youtube.com/watch?v=U2lLaxqugbU&t=...
302,ol,6,7,abone ol,V9Z155H7y7g,https://www.youtube.com/watch?v=V9Z155H7y7g&t=6s


In [53]:
df_word_group_time_loc_in_text_result.loc[0,"search_string"]

'ama benimle buraya kadar geldi sen şimdi git baba sonra beni'

In [54]:
text = df_word_group_time_loc_in_text_result.loc[13,"sentence"]
text

'sen nasıl bu kadar alçaldın küçücük bir çocuğu kullanacak hale geldin mi gerçekten gerçekleri de öğrenince aynı şeyleri düşünecek misin sen benim için buraya sadece intikam için gelen bir insansın çukurovadan sizi sileceğim dedi kimi kimi silecekmiş usta bizi ben sana zerre güvenmiyorum hala bir şeyler sakladığından da eminim senin baban beni sevmiyor biliyor musun çünkü ikimizde aynı kadına aşığız bu gece her şey bitecek gerçeği öğreneceksin'

In [55]:
words = re.findall(r"\w+", text, re.UNICODE)
words

['sen',
 'nasıl',
 'bu',
 'kadar',
 'alçaldın',
 'küçücük',
 'bir',
 'çocuğu',
 'kullanacak',
 'hale',
 'geldin',
 'mi',
 'gerçekten',
 'gerçekleri',
 'de',
 'öğrenince',
 'aynı',
 'şeyleri',
 'düşünecek',
 'misin',
 'sen',
 'benim',
 'için',
 'buraya',
 'sadece',
 'intikam',
 'için',
 'gelen',
 'bir',
 'insansın',
 'çukurovadan',
 'sizi',
 'sileceğim',
 'dedi',
 'kimi',
 'kimi',
 'silecekmiş',
 'usta',
 'bizi',
 'ben',
 'sana',
 'zerre',
 'güvenmiyorum',
 'hala',
 'bir',
 'şeyler',
 'sakladığından',
 'da',
 'eminim',
 'senin',
 'baban',
 'beni',
 'sevmiyor',
 'biliyor',
 'musun',
 'çünkü',
 'ikimizde',
 'aynı',
 'kadına',
 'aşığız',
 'bu',
 'gece',
 'her',
 'şey',
 'bitecek',
 'gerçeği',
 'öğreneceksin']

In [56]:
index_list = []
for i in range(len(words)):
    if words[i] in word_list:
        index_list.append(i)
    else:
        pass

In [57]:
all_index_list = []
var_index_list = []
for j in range(len(index_list)):
    try:
        var1 = index_list[j] + 1  
        var2 = index_list[j+1]
    except:
        var1 = index_list[j] + 1  
        var2 = index_list[-1]
    if var1 == var2:
        var3 = index_list[j]
        var_index_list.append(var3)
    else:
        var3 = index_list[j]
        var_index_list.append(var3)
        var4 = list(var_index_list)
        all_index_list.append(var4)
        var_index_list = []

In [58]:
all_index_list

[[0, 1, 2, 3],
 [6],
 [11, 12],
 [14],
 [16],
 [19, 20, 21, 22, 23, 24],
 [26],
 [28],
 [31],
 [38, 39, 40],
 [43, 44, 45],
 [47],
 [49],
 [51],
 [53, 54, 55],
 [57],
 [60, 61, 62, 63]]

In [59]:
sorted(all_index_list, key=len, reverse=True)

[[19, 20, 21, 22, 23, 24],
 [0, 1, 2, 3],
 [60, 61, 62, 63],
 [38, 39, 40],
 [43, 44, 45],
 [53, 54, 55],
 [11, 12],
 [6],
 [14],
 [16],
 [26],
 [28],
 [31],
 [47],
 [49],
 [51],
 [57]]

In [60]:
max(all_index_list, key=len)

[19, 20, 21, 22, 23, 24]

In [ ]:
text_list = []
for i in max(all_index_list, key=len):
    word = words[i]
    text_list.append(word)
text = " ".join(text_list)
text

In [4]:
# will test
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe. sentence is a string sentence. \n
    video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition \n
    sent_len_num is sentence occurs how many word at least. 
    '''
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        index_list = []

        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                    else:
                        pass
                else:
                   text = " ".join(text_list) 
            text_all_list.append(text)

            df_var = pd.DataFrame()
            for search_string in text_all_list:
                df_var["index"] = i
                df_var["search_string"] = search_string
                df_var["sentence"] = source_text
                df_var["video_id"] = opt_var
                df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [ ]:
exract_word_group(df_adjust_text_ratio, "sentence", "video_id", word_list, sent_len=False, sent_len_num=2)

In [61]:
text_all_list = []
for i in all_index_list:
    text_list = [] 
    for j in i:
        word = words[j]
        text_list.append(word)
        text = " ".join(text_list)
    text_all_list.append(text)

In [62]:
text_all_list

['sen nasıl bu kadar',
 'bir',
 'mi gerçekten',
 'de',
 'aynı',
 'misin sen benim için buraya sadece',
 'için',
 'bir',
 'sizi',
 'bizi ben sana',
 'hala bir şeyler',
 'da',
 'senin',
 'beni',
 'biliyor musun çünkü',
 'aynı',
 'bu gece her şey']

In [ ]:
text = df_videoid_sentence.loc[0,"sentence"]
text

In [ ]:
#text = "burcu biraz daha devam et devam et istanbul burcu burcu tamam oldu et izmir devam et devam et burcu burcu daha devam et burcu burcu"

In [ ]:
words = re.findall(r"\w+", text, re.UNICODE)
words

In [ ]:
len(words)

In [ ]:
words[1] in word_list

In [ ]:
index_list = []
for i in range(len(words)):
    if words[i] in word_list:
        index_list.append(i)
    else:
        pass

In [ ]:
index_list

In [ ]:
index_list[0] 

In [ ]:
len(index_list)

In [ ]:
all_index_list = []
var_index_list = []
for j in range(len(index_list)):
    try:
        var1 = index_list[j] + 1  
        var2 = index_list[j+1]
    except:
        var1 = index_list[j] + 1  
        var2 = index_list[-1]
    if var1 == var2:
        var3 = index_list[j]
        var_index_list.append(var3)
    else:
        var3 = index_list[j]
        var_index_list.append(var3)
        var4 = list(var_index_list)
        all_index_list.append(var4)
        var_index_list = []

In [ ]:
all_index_list

In [ ]:
all_index_list[0]

In [ ]:
max(all_index_list, key=len)

In [ ]:
text_list = []
for i in max(all_index_list, key=len):
    word = words[i]
    text_list.append(word)
text = " ".join(text_list)
text

In [ ]:
def exract_word_group(text, word_list):
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for i in max(all_index_list, key=len):
        word = words[i]
        text_list.append(word)
    text = " ".join(text_list)
    
    return text    

In [ ]:
text = "burcu biraz daha devam et devam et istanbul burcu burcu tamam oldu et izmir devam et devam et burcu burcu daha devam et burcu burcu"

In [ ]:
exract_word_group(text, word_list)

In [ ]:
df_adjust_sentence_ratio_test = df_adjust_sentence_ratio.head(100)

In [ ]:
df_adjust_sentence_ratio_test.loc[:,"search_string"] = df_adjust_sentence_ratio_test.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio_test

In [ ]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [ ]:
df_word_group_time_loc_result = word_group_time_loc(df_adjust_sentence_ratio_test, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result